Michał Zych, michal.zych96@gmail.com

Michał Warzecha, warzmich@gmail.com

In [1]:
%tensorflow_version 1.x
import keras
keras.__version__

TensorFlow 1.x selected.


Using TensorFlow backend.


'2.2.5'

In [8]:
from google.colab import files
uploaded = files.upload()
print(uploaded.keys())

Saving Bias_correction_ucl.csv to Bias_correction_ucl.csv
dict_keys(['Bias_correction_ucl.csv'])


In [12]:
import io
import pandas as pd


df2 = pd.read_csv(io.BytesIO(uploaded['Bias_correction_ucl.csv']))
# Dataset is now stored in a Pandas Dataframe

print(df2.keys)

# DELETE NaN VALUES
df = df2.dropna()
print(df.keys)

<bound method NDFrame.keys of       station        Date  Present_Tmax  ...  Solar radiation  Next_Tmax  Next_Tmin
0         1.0  2013-06-30          28.7  ...      5992.895996       29.1       21.2
1         2.0  2013-06-30          31.9  ...      5869.312500       30.5       22.5
2         3.0  2013-06-30          31.6  ...      5863.555664       31.1       23.9
3         4.0  2013-06-30          32.0  ...      5856.964844       31.7       24.3
4         5.0  2013-06-30          31.4  ...      5859.552246       31.2       22.5
...       ...         ...           ...  ...              ...        ...        ...
7747     23.0  2017-08-30          23.3  ...      4443.313965       28.3       18.1
7748     24.0  2017-08-30          23.3  ...      4438.373535       28.6       18.8
7749     25.0  2017-08-30          23.2  ...      4451.345215       27.8       17.4
7750      NaN         NaN          20.0  ...      4329.520508       17.4       11.3
7751      NaN         NaN          37.6  ...  

In [47]:
import numpy as np
data_set = df.to_numpy()


print(data_set.shape)
#DELETE DATE FROM DATASET (it's not a number)
X_part1 = data_set[:,:1]
X = data_set[:,2:23]
X = np.append(X, X_part1, axis=1)
y = data_set[:,23:]

X = np.array(X, np.float)
y = np.array(y, np.float)

print(X[0])

print(X.shape)
print(y.shape)

(7588, 25)
[2.87000000e+01 2.14000000e+01 5.82556877e+01 9.11163635e+01
 2.80741015e+01 2.30069362e+01 6.81888697e+00 6.94518053e+01
 2.33947471e-01 2.03895710e-01 1.61696854e-01 1.30928198e-01
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 3.76046000e+01 1.26991000e+02 2.12335000e+02 2.78500000e+00
 5.99289600e+03 1.00000000e+00]
(7588, 22)
(7588, 2)


In [0]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
train_data, test_data, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)


#NORMALIZATION DATA (USING ONLY TRAINING SET) 
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std
test_data -= mean
test_data /= std


Poniżej funkcja do wyznaczenia optymalnego kształu sieci neuronowej na bazie metody walidacji krzyżowej. Jako funkcje straty przyjęto MSE a jako metrykę MAE (średni bład bezwzględny)

In [0]:
from keras import models
from keras import layers
from sklearn.metrics import classification_report, confusion_matrix

def create_model(nr_elements, nr_hidden_layers, activation, input_shape):
  model = models.Sequential()
  model.add(layers.Dense(nr_elements, activation=activation, input_shape=(input_shape,)))
  for i in range(nr_hidden_layers-1):
    model.add(layers.Dense(nr_elements, activation=activation))
  model.add(layers.Dense(2))
  return model


def best_model_based_on_cross_validation(x_train, y_train, input_shape):
  len_validation_data = x_train.shape[0]/5
  max_accuracy =0
  best_nr_elems = 0
  best_nr_layer = 0
  len_training_data_cs = int(4* len_validation_data)
  min_correct = 10000
  for nr_elements in [32, 64, 128]:
    for nr_hidden_layers in [1,2,3]:
      model = create_model(nr_elements, nr_hidden_layers, 'relu', input_shape)
      model.compile(optimizer='rmsprop',
              loss='mse',
              metrics=['mae'])
      print("Nr elements: " + str(nr_elements) + ", nr hidden_layers: " + str(nr_hidden_layers))
      model.fit(x_train[0:len_training_data_cs],
                y_train[0:len_training_data_cs],
                epochs=20,
                batch_size=512,
                verbose=0)
      val_mse, val_mae = model.evaluate(x_train[len_training_data_cs:],
                                        y_train[len_training_data_cs:],
                                        verbose=0)
      print("mae value: " + str(val_mae))
      if val_mae < min_correct:
        min_correct = val_mae
        best_nr_elems = nr_elements
        best_nr_layer = nr_hidden_layers

  print("Best nr elems: {}, best nr hidden layers: {}".format(best_nr_elems, best_nr_layer))
  model = create_model(best_nr_elems, best_nr_layer, 'relu', input_shape) 
  model.compile(optimizer='rmsprop',
              loss='mse',
              metrics=['mae']) 
  model.fit(x_train,
            y_train,
            epochs=20,
            batch_size=512,
            verbose=0)
  return model
   

In [60]:
best_model = best_model_based_on_cross_validation(train_data, y_train, train_data.shape[1])

Nr elements: 32, nr hidden_layers: 1
mae value: 18.296536225311517
Nr elements: 32, nr hidden_layers: 2
mae value: 4.6836771928062735
Nr elements: 32, nr hidden_layers: 3
mae value: 2.5479853212371353
Nr elements: 64, nr hidden_layers: 1
mae value: 14.319753014024839
Nr elements: 64, nr hidden_layers: 2
mae value: 2.211879469627558
Nr elements: 64, nr hidden_layers: 3
mae value: 1.8681989828745524
Nr elements: 128, nr hidden_layers: 1
mae value: 6.694445854748866
Nr elements: 128, nr hidden_layers: 2
mae value: 1.7030972682228385
Nr elements: 128, nr hidden_layers: 3
mae value: 1.8272562631340914
Best nr elems: 128, best nr hidden layers: 2


In [64]:
test_mse_score, test_mae_score = best_model.evaluate(test_data, y_test)
print(test_mae_score)

mean_24 = np.mean(y_test[0])
mean_25 = np.mean(y_test[1])
sum_mean = mean_24 + mean_25
print(mean_24)
print(mean_25)
print(sum_mean)


print("Sredni blad wzgledny: " + str(test_mae_score/sum_mean))

1139/1139 [==============================] - 0s 38us/step
1.6640877167942023
26.6
24.65
51.25
Sredni blad wzgledny: 0.03247000423013078
